In [61]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

import sys

# Append the path to Pydoop to sys.path
#sys.path.append("/usr/local/lib/python3.8/dist-packages")

Spark Configuration

In [43]:
# Spark Session - 2 CORE - imporvement CONFIGURATION!!
# spark_session = SparkSession.builder\
#     .master("local[2]")\
#     .appName("pseudo_spark_nora")\
#     .getOrCreate()

# Set Hadoop configuration directory
#os.environ['HADOOP_CONF_DIR'] = '/path/to/hadoop/conf'
#spark_session.stop()
spark_session = SparkSession.builder \
    .appName("HDFS_Connection_Test_nora") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://master-node:9000") \
    .config("spark.jars.packages", "LLNL:spark-hdf5:0.0.4") \
    .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

24/03/10 11:04:20 INFO SparkContext: Running Spark version 3.5.1
24/03/10 11:04:20 INFO SparkContext: OS info Linux, 5.4.0-167-generic, amd64
24/03/10 11:04:20 INFO SparkContext: Java version 1.8.0_392
24/03/10 11:04:20 INFO ResourceUtils: ==============================================================
24/03/10 11:04:20 INFO ResourceUtils: No custom resources configured for spark.driver.
24/03/10 11:04:20 INFO ResourceUtils: ==============================================================
24/03/10 11:04:20 INFO SparkContext: Submitted application: HDFS_Connection_Test_nora
24/03/10 11:04:20 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/03/10 11:04:20 INFO ResourceProfile: Limiting resource is cpus at 1 tasks per executor
24/03/10 11:04

File Path

In [44]:
#file_roots = ['hdfs://130.238.29.183:9000//input/millionsongsubset.tar.gz']
#directory_path = "data/B/A/A"
#directory_path = "data_AZ"
#Path300
#directory_path = "/user/hadoop/MillionSongSubset/A/A/"
#directory_path = "/user/hadoop/MillionSongSubset/A/A/A"
directory_path = "hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A"

In [ ]:
# # Use Spark to list files in HDFS directory
# hdfs_files = spark_session._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
#     .listStatus(spark_session._jvm.org.apache.hadoop.fs.Path(directory_path))

# 1) Saving the paths to the HDF5 files

In [45]:
def get_h5_file_paths_hdfs(spark, hdfs_directory_path):
    h5_file_paths = []

    # Use Spark to list files in HDFS directory
    hdfs_files = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
        .listStatus(spark._jvm.org.apache.hadoop.fs.Path(hdfs_directory_path))

    # Extract file paths from HDFS file status
    for hdfs_file_status in hdfs_files:
        file_path = hdfs_file_status.getPath().toString()

        if hdfs_file_status.isDirectory():
            # If it's a directory, recursively gather paths
            h5_file_paths.extend(get_h5_file_paths_hdfs(spark, file_path))
        elif file_path.endswith(".h5"):
            # If it's an HDF5 file, add to the list
            h5_file_paths.append(file_path)

    return h5_file_paths

In [47]:
h5_file_paths = get_h5_file_paths_hdfs(spark_session, directory_path)

print('{} HDF5 file paths gathered (HDFS)'.format(len(h5_file_paths)))
for file_path in h5_file_paths[:50]:
    print(file_path)

11 HDF5 file paths gathered (HDFS)
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAABD128F429CF47.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAADZ128F9348C2E.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAEF128F4273421.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAFD128F92F423A.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMO128F1481E7F.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMQ128F1460CD3.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAPK128E0786D96.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAARJ128F9320760.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVG12903CFA543.h5
hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVO128F93133D4.h5


# 2) List h5_file_paths is converted to RDD

In [48]:
h5_file_paths[:30]

['hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAABD128F429CF47.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAADZ128F9348C2E.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAEF128F4273421.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAFD128F92F423A.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMO128F1481E7F.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAMQ128F1460CD3.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAPK128E0786D96.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAARJ128F9320760.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVG12903CFA543.h5',
 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAVO128F93133D4.h5']

In [49]:
file_paths = spark_context.parallelize(h5_file_paths)
file_paths

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [50]:
file_first = file_paths.first()
file_first

24/03/10 11:05:20 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/10 11:05:20 INFO DAGScheduler: Got job 0 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/10 11:05:20 INFO DAGScheduler: Final stage: ResultStage 0 (runJob at PythonRDD.scala:181)
24/03/10 11:05:20 INFO DAGScheduler: Parents of final stage: List()
24/03/10 11:05:20 INFO DAGScheduler: Missing parents: List()
24/03/10 11:05:20 INFO DAGScheduler: Submitting ResultStage 0 (PythonRDD[1] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/10 11:05:20 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 5.9 KiB, free 366.3 MiB)
24/03/10 11:05:20 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 3.8 KiB, free 366.3 MiB)
24/03/10 11:05:20 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on master-node:42965 (size: 3.8 KiB, free: 366.3 MiB)
24/03/10 11:05:20 INFO SparkContext: Created broadcast 0 from broadcast at

'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5'

# 3) H5Py file object file creation from the H5 file 

In [15]:
from pydoop import hdfs

def extract_info_pyd(hdfs_path):
    with hdfs.open(hdfs_path) as f:
        with h5py.File(f, driver='fileobj') as h:
            # Assuming the dataset path is '/analysis/songs'
            dataset = h['/analysis/songs']
            np_dataset = np.array(dataset[0])
            
            # Extracting selected features
            selected_features = np_dataset[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
            
            return selected_features.tolist()

In [51]:
def extract_info(file_path):
    with h5py.File(file_path, 'r') as file:
        song = file['analysis']['songs']
        np_song = np.array(song[0])
        selected_features = np_song[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
        return selected_features.tolist()
        #print(f"File found: {file_path}")
         #return file_path
    # except FileNotFoundError:
    #     print(f"File not found: {file_path}")
    #     return None

In [52]:
#file = h5py.File(file_first)
h5_file_paths = ['hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5']
h5_file_path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5'
file_paths = spark_context.parallelize(h5_file_paths)
files_infos_rdd = file_paths.map(extract_info)
#files_infos_rdd = file_paths.map(extract_info)

In [53]:
file_paths_df = spark_session.createDataFrame([h5_file_path], "string").toDF("file_path")

24/03/10 11:05:33 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/10 11:05:33 INFO SharedState: Warehouse path is 'file:/home/hadoop/MSD_Project/DataEngineeringProject/Basic%20analysis%20hdf5_to_rdd/spark-warehouse'.
24/03/10 11:05:33 INFO ServerInfo: Adding filter to /SQL: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/10 11:05:33 INFO ServerInfo: Adding filter to /SQL/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/10 11:05:33 INFO ServerInfo: Adding filter to /SQL/execution: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/10 11:05:33 INFO ServerInfo: Adding filter to /SQL/execution/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/10 11:05:33 INFO ServerInfo: Adding filter to /static/sql: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter


In [54]:
# Register the extract_info function as a Spark UDF
spark_session.udf.register("extract_info", extract_info)

<function __main__.extract_info(file_path)>

In [55]:
result_df = file_paths_df.withColumn("result", expr("extract_info(file_path)"))

In [56]:
# Collect and print the result
result_collected = result_df.collect()
for row in result_collected:
    print(row.result)

24/03/10 11:05:42 INFO SparkContext: Starting job: collect at /tmp/ipykernel_289261/897956410.py:2
24/03/10 11:05:42 INFO DAGScheduler: Got job 1 (collect at /tmp/ipykernel_289261/897956410.py:2) with 2 output partitions
24/03/10 11:05:42 INFO DAGScheduler: Final stage: ResultStage 1 (collect at /tmp/ipykernel_289261/897956410.py:2)
24/03/10 11:05:42 INFO DAGScheduler: Parents of final stage: List()
24/03/10 11:05:42 INFO DAGScheduler: Missing parents: List()
24/03/10 11:05:42 INFO DAGScheduler: Submitting ResultStage 1 (MapPartitionsRDD[12] at collect at /tmp/ipykernel_289261/897956410.py:2), which has no missing parents
24/03/10 11:05:42 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 26.3 KiB, free 366.3 MiB)
24/03/10 11:05:42 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 12.3 KiB, free 366.3 MiB)
24/03/10 11:05:42 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on master-node:42965 (size: 12.3 KiB, free

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_289261/1347997957.py", line 2, in extract_info
  File "/usr/local/lib/python3.8/dist-packages/h5py/_hl/files.py", line 562, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.8/dist-packages/h5py/_hl/files.py", line 235, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 102, in h5py.h5f.open
FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


24/03/10 11:05:45 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 1) (worker-node-1 executor 1): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 1 in stage 1.0 failed 4 times, most recent failure: Lost task 1.3 in stage 1.0 (TID 5) (worker-node-2 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_289261/1347997957.py", line 2, in extract_info
  File "/usr/local/lib/python3.8/dist-packages/h5py/_hl/files.py", line 562, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.8/dist-packages/h5py/_hl/files.py", line 235, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 102, in h5py.h5f.open
FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open f

In [30]:
print(files_infos_rdd.collect())

[None]


24/03/10 09:24:57 INFO SparkContext: Starting job: collect at /tmp/ipykernel_283650/3743736607.py:1
24/03/10 09:24:57 INFO DAGScheduler: Got job 6 (collect at /tmp/ipykernel_283650/3743736607.py:1) with 2 output partitions
24/03/10 09:24:57 INFO DAGScheduler: Final stage: ResultStage 6 (collect at /tmp/ipykernel_283650/3743736607.py:1)
24/03/10 09:24:57 INFO DAGScheduler: Parents of final stage: List()
24/03/10 09:24:57 INFO DAGScheduler: Missing parents: List()
24/03/10 09:24:57 INFO DAGScheduler: Submitting ResultStage 6 (PythonRDD[9] at collect at /tmp/ipykernel_283650/3743736607.py:1), which has no missing parents
24/03/10 09:24:57 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 6.0 KiB, free 366.2 MiB)
24/03/10 09:24:57 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 3.9 KiB, free 366.2 MiB)
24/03/10 09:24:57 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on master-node:45719 (size: 3.9 KiB, free: 366.3

PARQUET

In [57]:
h5_file_path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5'

In [58]:
df = spark_session.read.format("com.spark.hdf5") \
    .option("path", h5_file_path) \
    .load()

Py4JJavaError: An error occurred while calling o352.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.spark.hdf5. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:172)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	

In [38]:
store = pd.HDFStore(h5_file_path,  mode='r', driver='pytables')

ValueError: Invalid or not supported driver: 'pytables'

In [22]:
print(spark_session._jsc.hadoopConfiguration())

Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, resource-types.xml, hdfs-default.xml, hdfs-rbf-default.xml, hdfs-site.xml, hdfs-rbf-site.xml, __spark_hadoop_conf__.xml


In [15]:
h5py.run_tests()

Tests require pytest, pytest not installed


1

# 4) Create Spark Data Frame

In [61]:
# Definiere das Schema für den DataFrame
schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("duration", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("loudness", FloatType(), True),
])

In [62]:
spark_df = sqlContext.createDataFrame(files_infos_rdd.flatMap(lambda x: x), schema=schema)

24/03/08 14:24:48 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/08 14:24:48 INFO SharedState: Warehouse path is 'file:/home/hadoop/MSD_Project/DataEngineeringProject/Basic%20analysis%20hdf5_to_rdd/spark-warehouse'.
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL/execution: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /SQL/execution/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:48 INFO ServerInfo: Adding filter to /static/sql: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/08 14:24:49 INFO BlockManagerInfo: Removed broadcast_2_piece0 on master-node:43017 in memory

In [63]:
# Zeige den DataFrame-Inhalt
spark_df.show()

24/03/08 14:24:56 INFO CodeGenerator: Code generated in 283.099102 ms
24/03/08 14:24:56 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/08 14:24:56 INFO DAGScheduler: Got job 3 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/08 14:24:56 INFO DAGScheduler: Final stage: ResultStage 3 (showString at NativeMethodAccessorImpl.java:0)
24/03/08 14:24:56 INFO DAGScheduler: Parents of final stage: List()
24/03/08 14:24:56 INFO DAGScheduler: Missing parents: List()
24/03/08 14:24:56 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[9] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/08 14:24:56 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 22.5 KiB, free 366.3 MiB)
24/03/08 14:24:56 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 10.4 KiB, free 366.3 MiB)
24/03/08 14:24:56 INFO BlockManagerInfo: Added broadcast_

Py4JJavaError: An error occurred while calling o62012.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 13) (worker-node-2 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0037/container_1709508717695_0037_01_000002/pyspark.zip/pyspark/worker.py", line 1227, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0037/container_1709508717695_0037_01_000002/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1709508717695_0037/container_1709508717695_0037_01_000002/pyspark.zip/pyspark/

# Calculation (Average)

In [15]:
average_values = spark_df.agg({'danceability': 'avg', 'duration': 'avg', 'tempo': 'avg', 'energy': 'avg', 'loudness': 'avg'}).collect()[0]
print("Average Values::")
for col_name, avg_value in average_values.asDict().items():
    print(f"{col_name}: {avg_value}")

24/03/01 12:54:24 INFO CodeGenerator: Code generated in 159.97677 ms
24/03/01 12:54:24 INFO DAGScheduler: Registering RDD 10 (collect at /tmp/ipykernel_56411/3998951206.py:1) as input to shuffle 0
24/03/01 12:54:24 INFO DAGScheduler: Got map stage job 4 (collect at /tmp/ipykernel_56411/3998951206.py:1) with 2 output partitions
24/03/01 12:54:24 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (collect at /tmp/ipykernel_56411/3998951206.py:1)
24/03/01 12:54:24 INFO DAGScheduler: Parents of final stage: List()
24/03/01 12:54:24 INFO DAGScheduler: Missing parents: List()
24/03/01 12:54:24 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[10] at collect at /tmp/ipykernel_56411/3998951206.py:1), which has no missing parents
24/03/01 12:54:24 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 36.3 KiB, free 413.9 MiB)
24/03/01 12:54:24 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 15.5 KiB, free 413.9 MiB)
24

Average Values::
avg(duration): 236.8841377784466
avg(tempo): 125.78977758234197
avg(energy): 0.0
avg(danceability): 0.0
avg(loudness): -10.435855814655746


# STOP SPARK

In [59]:
# release the cores
spark_context.stop()

24/03/10 11:07:45 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/03/10 11:07:45 INFO SparkUI: Stopped Spark web UI at http://master-node:4040
24/03/10 11:07:45 INFO YarnClientSchedulerBackend: Interrupting monitor thread
24/03/10 11:07:45 INFO YarnClientSchedulerBackend: Shutting down all executors
24/03/10 11:07:45 INFO YarnSchedulerBackend$YarnDriverEndpoint: Asking each executor to shut down
24/03/10 11:07:45 INFO YarnClientSchedulerBackend: YARN client scheduler backend Stopped
24/03/10 11:07:45 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/03/10 11:07:45 INFO MemoryStore: MemoryStore cleared
24/03/10 11:07:45 INFO BlockManager: BlockManager stopped
24/03/10 11:07:45 INFO BlockManagerMaster: BlockManagerMaster stopped
24/03/10 11:07:45 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/03/10 11:07:45 INFO SparkContext: Successfully stopped SparkContext
